In [ ]:
from pathways import Pathways
p = Pathways(
    datapackage="/Users/romain/GitHub/ZA_cobenefits/remind-SSP2-NPi-stem-WEM.zip",
    debug=False,
    activities_mapping="act_categories_agg.yaml",
    ecoinvent_version="3.11"
)

In [ ]:
import numpy as np
p.calculate(
    methods=[m for m in p.lcia_methods if "EF v3.1 -" in str(m)][-5:],
    #edges_methods=[
    #    ('GeoPolRisk', 'paired', '2024'),
        #('AWARE 2.0', 'Country', 'all', 'yearly'),
        #('AWARE 2.0', 'Country', 'all', 'yearly'),
    #],
    #regions=[r for r in p.scenarios.coords["region"].values if r not in ("World",)],
    regions=["ZA",],
    #scenarios=[
    #    #"remind - SSP2-Base - Sustainable development - remind - SPS4",
    #    "SSP2-Base - Business As Usual - remind",
    #    "SSP2-PkBudg500 - Sustainable development - remind"
    #    
    #],
    variables=[v for v in list(p.scenarios.variables.values) if v.startswith("EXT")],
    multiprocessing=False,
    #variables = ['hydrogen, from natural gas'],
    #years=[
    #    2020,
        #2030
    #],
    #demand_cutoff=0.01,
)

In [ ]:
p.scenarios.sel(variables="FE - final energy - Industry - Non-energy use - Bio CH4").attrs["units"]["FE - final energy - Industry - Non-energy use - Bio CH4"]

In [ ]:
from pathways import Pathways
p = Pathways(
    datapackage="/Users/romain/GitHub/premise/dev/pathways_2025-10-22/datapackage.json",
    debug=False,
    activities_mapping="act_categories_agg.yaml",
    ecoinvent_version="3.11"
)

import numpy as np
p.calculate(
    methods=[m for m in p.lcia_methods if "EF v3.1 -" in str(m)][:2],
    #edges_methods=[
    #    ('AWARE 2.0', 'Country', 'all', 'yearly'),
    #],
    #regions=[r for r in p.scenarios.coords["region"].values if r not in ("World",)],
    regions=["World",],
    #scenarios=[
    #    #"remind - SSP2-Base - Sustainable development - remind - SPS4",
    #    "SSP2-Base - Business As Usual - remind",
    #    "SSP2-PkBudg500 - Sustainable development - remind"
    #    
    #],
    variables=[v for v in list(p.scenarios.variables.values) if v.startswith("FE")][:5],
    multiprocessing=False,
    #variables = ['hydrogen, from natural gas'],
    years=[
        2020,
        2030
    ],
    #demand_cutoff=0.01,
)

In [ ]:
p.export_results()

In [ ]:
import pandas as pd
df = pd.read_parquet("results_20251114_111902.gzip")

In [ ]:
len(df)

In [ ]:
df = df[df["value"]!=0.0]
df = df[~df["value"].isnull()]
df=df.reset_index()

In [ ]:
len(df)

In [ ]:
from premise.geomap import Geomap
geo = Geomap("remind")

for region in [r[-1] for r in list(geo.geo.keys()) if r[0] == "REMIND"]:
    if region == "World":
        continue
        
    REGION = geo.iam_to_ecoinvent_location(region)
    REGION.append(region)
    df.loc[df["location"].isin(REGION), "location"] = region

remaining_locs = {
    'AU-NSW': 'CAZ',
    'AU-QLD': 'CAZ',
    'AU-SA': 'CAZ',
    'AU-TAS': 'CAZ',
    'AU-VIC': 'CAZ',
    'AU-WA': 'CAZ',
    #'GLO': 'World',
    'IAI Area, Gulf Cooperation Council': 'MEA',
    #'RoW': 'World',
    'US-CO': 'USA',
    'US-IA': 'USA',
    'US-ID': 'USA',
    'US-IL': 'USA',
    'US-IN': 'USA',
    'US-MN': 'USA',
    'US-ND': 'USA',
    'US-NE': 'USA',
    'US-OH': 'USA',
    'US-OR': 'USA',
    'US-SD': 'USA',
    'US-WA': 'USA',
    'US-WI': 'USA',
    'UN-EASIA': 'OAS',
}

for k, v in remaining_locs.items():
    df.loc[df["location"] == k, "location"] = v

df = df.groupby(
    [
        'variable',
        'year',
        'region',
        'model',
        'scenario',
        'impact_category',
        'location',
        'act_category'
    ]).sum().reset_index()

In [ ]:
len(df)

In [ ]:
df["location"].unique()

In [ ]:
df.head()

In [ ]:
df.groupby(["act_category",]).sum(numeric_only=True).sort_values(by="value", ascending=False)

In [ ]:
df["act_category"].unique()

In [ ]:
len(df)

In [ ]:
from pivottablejs import pivot_ui
from IPython.display import HTML
pivot_ui(df, outfile_path='pivottable_Nyle.html')

In [ ]:
import pygwalker as pyg

In [ ]:
walker = pyg.walk(df)

In [ ]:
import pygwalker as pyg
import os, webbrowser

# df = da.to_dataframe(name="value").reset_index()

ret = pyg.walk(df, return_html=True)      # may return a PygWalker object
html = ret if isinstance(ret, str) else ret.to_html()

path = os.path.abspath("pygwalker_dashboard.html")
with open(path, "w", encoding="utf-8") as f:
    f.write(html)

webbrowser.open("file://" + path)


In [ ]:
walter = pyg.walk(df, env='Streamlit') 

In [ ]:
walter.